# Global Centroid Moment Tensor 3D (GCMT3D)

## The workflow

Make sure you are in the right virtual environment.


### Creating the Database Skeleton

The `DatabaseSkeleton` `class` handles the setup of the database 
and creates new directories for new CMTs.

```bash
database
├── eq_<eq_ID_1>
│   ├── CMT_SIMs/
│   │   ├── CMT_depth/
│   │   │    ├── DATA/
│   │   │    ├── DATABASES_MPI/
│   │   │    ├── OUTPUFILES/
│   │   │    └── bin/        # symbolic link --> SPECFEM_DIR/bin/
│   │   ├── CMT_lat/
│   │   │    ├── DATA/
│   │   │    :
│   │   ├── CMT_lon/
│   │   ├── CMT_pp/
│   │   ├── CMT_rp/
│   │   ├── CMT_rr/
│   │   ├── CMT_rt/
│   │   ├── CMT_tp/
│   │   └── CMT_tt/
│   ├── eq_9703873.cmt
│   ├── responses/
│   └── seismograms/
│
├── eq_<eq_ID_2>/
│   ├── CMT_SIMs/
│   :
│
├── eq_<eq_ID_3>/
:

```

In [1]:
from gcmt3d.data.management.skeleton import DataBaseSkeleton


# Set database directory
databasedir = "data/sample_database"
cmt_filename = "data/CMTSOLUTION"

# Create Database skeleton class. Note that the specfem directory is not set
# because it is not available on every system.
DB = DataBaseSkeleton( basedir=databasedir, cmt_fn=cmt_filename, specfem_dir=None, verbose=True)

# Create database structure
DB.create_all()


### Downloading the Data
The first step in the inversion process is downloading data that corresponds to the original solution of Global CMT Catalogue, which is maintained by the Lamont-Doherty Earth Observatory.

To do this we use the Specifically design DataRequest class. We have to define certain parameters before we download the data:

In [2]:
from gcmt3d.data import DataRequest

# Parameters
duration = 3600      # seconds, 1h = 3600s
channels = ['BHN','BHE','BHZ']   # needs to be a list
locations=['00']     # location 00 is the primary location for most seismometers
starttime_offset=-60 # -60 seconds is 1 min before the earthquakes inverted cmt solution 
resp_format="resp"   # response format


# Earthquake ID
eqID = "eq_060994A"
outputdir="data/"+eqID+"/"  # specific output directory here left open if left open files will 
                            # be saved in the folder wher the `.cmt` file is located


# Earthquake and Station parameters
cmt_filename = "data/" + eqID + "/earthquake_data/CMTSOLUTION"
stationlist_filename = "./src/gcmt3d/data/download/resources/stations.txt"

# Create Request Object   
Request = DataRequest.from_file(cmt_filename,
                                stationlist_filename,
                                duration=duration,
                                channels=channels,
                                locations=locations,
                                starttime_offset=starttime_offset,
                                outputdir=outputdir)

# Print Earthquake Download Info
print(Request)

FileNotFoundError: [Errno 2] No such file or directory: 'data/eq_060994A/earthquake_data/CMTSOLUTION'

After setting up the Request and inspecting the parameters we can download the data:

In [3]:
# Download the data using the download method
Request.download()

Reading data selection from list file 'data/eq_060994A//request.txt'
Received 1.3 KB of metadata in 0.5 seconds (2.5 KB/s)
Processed metadata for 45 channel epochs in 0.5 seconds
Received 4.6 MB of time series data in 5.4 seconds (864.8 KB/s) - written to data/eq_060994A/seismograms/obs/060994A.mseed
Received 114.0 KB of StationXML in 0.6 seconds (184.1 KB/s)
DONE at 2019-04-08 01:40:42


The next step is to create a STATIONS file, which tells specfem3d_globe where to "record" the simulated traces:

In [4]:
# Creating STATIONS file
Request.specfem_list()


STATIONS FILE WRITTEN.



## Setting up specfem3d_globe

To run specfem in an automated fashion, we need the derivative information of the traces with respect to the moment tensor attributes as well as the centroid location.
I import the values that Wenjie set in his inversion.


In [3]:
from pycmt3d.constant import SCALE_MOMENT, SCALE_DEPTH, SCALE_LONGITUDE, SCALE_LATITUDE
from gcmt3d.source import CMTSource
from gcmt3d.data import SpecfemSources
import os


# Basic parameters
cmt = CMTSource.from_CMTSOLUTION_file(cmt_filename)
npar = 9
dm = SCALE_MOMENT       # 10**22 dyne*cm
dx = SCALE_DEPTH        # 1000 m
ddeg = SCALE_LATITUDE   # 0.001 deg
outdir = os.path.join(outputdir,"earthquake_data")

sfsource = SpecfemSources(cmt, npar=npar, dm=dm, dx=dx, ddeg=ddeg,
                          outdir=outdir)
sfsource.write_sources()
